In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import sys
import os
from PIL import Image
import pickle

In [3]:
def ims_to_mask(im_original, im_out):
    mask = np.abs(im_original - im_out)
    mask = cv2.resize(mask,(96,96))
    
    background = np.zeros((450,800,3))
    x_offset=350
    y_offset=125
    background[y_offset:y_offset+mask.shape[0], x_offset:x_offset+mask.shape[1]] = mask
    
    kernel = np.ones((5,5))

    dilated = (cv2.dilate(background, kernel)*255).astype(int)
    
    dilated_gray = np.average(dilated, axis = 2).astype(int)
    
    return background, dilated, dilated_gray

In [4]:
def load_depth_map(im, k = 0.5):
    iminv = 255 - im
    out = iminv - k * (255 - iminv)
    out[np.where(out <= 0)] = 0
    
    return out
    

In [5]:
def projection(depthmap, dilated_gray, w = 800, h = 450, focalLength = 90.0, 
               scalingFactor = 1500, mask_offset_k = 0.4):
    
    centerX = w / 2.0
    centerY = h / 2.0

    Xrang = np.arange(0,w,1)
    Yrang = np.arange(0,h,1)
    imXY = np.meshgrid(Xrang, Yrang)
    
    zmask = depthmap.copy()
    zmask[np.where(dilated_gray < 13)] = 0
    
    zmask_offset = dilated_gray.copy()
    zmask_offset[np.where(zmask == 0)] = 0
    
    Z = zmask / scalingFactor + zmask_offset * mask_offset_k / scalingFactor
    
    X = (imXY[0] - centerX) * Z / focalLength
    Y = (imXY[1] - centerY) * Z / focalLength

    Xworld = np.vstack((X.flatten(), Y.flatten(), Z.flatten())).T
    
    X = Xworld.reshape(h, w, 3)
    
    return Xworld, X

In [6]:
def get_mesh(X, Xflat):
    
    triangles = []
    uv = []
    vertices = []
    count = 0
    maps = {}
    
    for i in range(X.shape[0] - 1):

        for j in range(X.shape[1]):

            curr = (i)*X.shape[1]+j
            right = (i)*X.shape[1]+j+1
            below = (i+1)*X.shape[1]+j
            left = (i+1)*X.shape[1]+j-1
            black = [0,0,0]
            
            curr_notblack = np.linalg.norm(Xflat[curr] - black) != 0
            right_notblack = np.linalg.norm(Xflat[right] - black) != 0
            below_notblack = np.linalg.norm(Xflat[below] - black) != 0
            left_notblack = np.linalg.norm(Xflat[left] - black) != 0
            
            if curr_notblack:
                vertices.append(Xflat[curr])
                uv.append([i,j])
                maps[curr] = count
                count += 1

            

            if j is 0:
                if curr_notblack and right_notblack and below_notblack:
                    triangles.append([curr, right,below ])
            elif j is X.shape[1]-1:
                if curr_notblack and below_notblack and left_notblack:
                    triangles.append([curr, below, left])
            else:
                if curr_notblack and right_notblack and below_notblack:
                    triangles.append([curr, right,below])
                if curr_notblack and below_notblack and left_notblack:
                    triangles.append([curr, below, left])
    
    i = X.shape[0] - 1
    for j in range(X.shape[1]):
        curr = (i)*X.shape[1]+j
        if curr_notblack:
                vertices.append(Xflat[curr])
                uv.append([i,j])
    
    
    
    return triangles, vertices, uv , maps

In [7]:
def write_mesh_csv(triangles, vertices, uv, index_maps, file_epoch, file_index):
    
    trgles = pd.DataFrame(triangles, columns = ['one', 'two', 'three'])

    for row in trgles.iterrows():
        row[1]['one'] = maps[row[1]['one']]
        row[1]['two'] = maps[row[1]['two']]
        row[1]['three'] = maps[row[1]['three']]
    
    vts = np.array(vertices)
    uvs = np.array(uv)
    
    points = pd.DataFrame(vts, columns = ['x','y','z'])
    uvdefs = pd.DataFrame(uvs, columns = ['u','v'])
    
    if not os.path.isdir(str(file_epoch)):
        os.mkdir(str(file_epoch))
        
    direct = os.path.join(str(file_epoch), 'mask' + str(file_index))
    
    if not os.path.isdir(direct):
        os.mkdir(direct)
    
    triangle_file_path = os.path.join(direct, 'triangles' + str(file_index) +'.csv')
    points_file_path = os.path.join(direct, 'points' + str(file_index) +'.csv')
    uvs_file_path = os.path.join(direct, 'uvs' + str(file_index) +'.csv')
    
    trgles.to_csv(triangle_file_path)
    points.to_csv(points_file_path)
    uvdefs.to_csv(uvs_file_path)


In [8]:
background, dilated, dilated_gray = ims_to_mask(original[31], altered[31])

imz = cv2.imread('data/1z.tif', 0)
depth = load_depth_map(imz)

Xworld, X = projection(depth, dilated_gray)

triangles, vertices, uv , maps = get_mesh(X, Xworld)

file_epoch = 1
file_index = 1

write_mesh_csv(triangles, vertices, uv, maps, file_epoch, file_index)

NameError: name 'original' is not defined

In [31]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.figure(figsize=(16,16))
ax = plt.axes(projection='3d')
ax.scatter(Xworld[:,0],Xworld[:,1],Xworld[:,2])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Recovered World Points')

Text(0.5, 0.92, 'Recovered World Points')

In [ ]:
# target image indices
indices1 = list(range(80,109))

im_idx = [31, 60, 77] + indices1

for k in range(3,11):
    path_original = os.path.join('meshwrite/pickles', 'batch_fi' + str(k) + '.pkl')
    path_altered = os.path.join('meshwrite/pickles', 'pert_fi' + str(k) + '.pkl')
    original = pickle.load(open(path_original, 'rb'))
    altered = pickle.load(open(path_altered, 'rb'))
    
    for i in im_idx:
        background, dilated, dilated_gray = ims_to_mask(original[i], altered[i])

        imz = cv2.imread('data/1z.tif', 0)
        depth = load_depth_map(imz)

        Xworld, X = projection(depth, dilated_gray)

        triangles, vertices, uv , maps = get_mesh(X, Xworld)

        write_mesh_csv(triangles, vertices, uv, maps, k, i)
    

In [ ]:
background = cv2.imread()

In [9]:
imz = cv2.imread('data/1z.tif', 0)
depth = load_depth_map(imz)

plt.imshow(depth, cmap = 'gray')

In [14]:
#get file destination folders

folders = os.listdir('C:\\Users\\airla\\Intro_to_comp_final\\meshes\\MeshCSV')

folders = [i for i in folders if i.isdigit()]

folders

['10', '3', '4', '5', '6', '7', '8', '9']

In [15]:
def ims_to_mask2(im_original, im_out, resize = 96, dim = 200, ker_size = 3):
    mask = np.abs(im_original - im_out)
    mask = cv2.resize(mask,(resize, resize))
    
    background = np.zeros((dim,dim,3))
    x_offset= int((dim - resize)/2)
    y_offset= int((dim - resize)/2)

    background[y_offset:y_offset+mask.shape[0], x_offset:x_offset+mask.shape[1]] = mask
    
    kernel = np.ones((ker_size,ker_size))

    dilated = (cv2.dilate(background, kernel)*255).astype(int)
    
    dilated_gray = np.average(dilated, axis = 2).astype(int)
    
    return background, dilated, dilated_gray



In [64]:
indices1 = list(range(80,109))

im_idx = [31, 60, 77] + indices1
fig, ax = plt.subplots(32, 8, figsize = (17,70))

for k in range(3,11):
    path_original = os.path.join('meshwrite/pickles', 'batch_fi' + str(k) + '.pkl')
    path_altered = os.path.join('meshwrite/pickles', 'pert_fi' + str(k) + '.pkl')
    original = pickle.load(open(path_original, 'rb'))
    altered = pickle.load(open(path_altered, 'rb'))
    
    for index, i in enumerate(im_idx):
        _,_, dilated_gray = ims_to_mask2(original[i], altered[i])
        ax[index][k-3].imshow(dilated_gray, cmap = 'gray')
        ax[index][k-3].axis(False)

plt.savefig('phantom_gray2.png')

In [63]:
indices1 = list(range(80,109))

im_idx = [31, 60, 77] + indices1
fig, ax = plt.subplots(32, 8, figsize = (17,70))

for k in range(3,11):
    path_original = os.path.join('meshwrite/pickles', 'batch_fi' + str(k) + '.pkl')
    path_altered = os.path.join('meshwrite/pickles', 'pert_fi' + str(k) + '.pkl')
    original = pickle.load(open(path_original, 'rb'))
    altered = pickle.load(open(path_altered, 'rb'))
    
    for index, i in enumerate(im_idx):
        _,dilated,_ = ims_to_mask2(original[i], altered[i])
        ax[index][k-3].imshow(dilated)
        ax[index][k-3].axis(False)

plt.savefig('phantom_color.png')

In [37]:
indices1 = list(range(80,109))

im_idx = [31, 60, 77] + indices1

for k in range(3,11):
    path_original = os.path.join('meshwrite/pickles', 'batch_fi' + str(k) + '.pkl')
    path_altered = os.path.join('meshwrite/pickles', 'pert_fi' + str(k) + '.pkl')
    original = pickle.load(open(path_original, 'rb'))
    altered = pickle.load(open(path_altered, 'rb'))
    
    for index, i in enumerate(im_idx):
        _,dilated, dilated_gray = ims_to_mask2(original[i], altered[i])
        
#         img = np.array(dilated, dtype=np.uint8)
#         hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#         value = 200
#         cv2.add(hsv[:,:,2], value, hsv[:,:,2])
#         dilated = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        dilated = (dilated * 1.4).astype(int)
        dilated_gray = (dilated_gray * 1.4).astype(int)
        
        gray_path = os.path.join('MeshCSV', str(k),str(k)+'_'+str(i)+'_gray'+'.png')
        color_path = os.path.join('MeshCSV', str(k),str(k)+'_'+str(i)+'_color'+'.png')

        cv2.imwrite(gray_path, dilated_gray)
        cv2.imwrite(color_path, dilated)


In [53]:
background, dilated, dilated_gray = ims_to_mask2(original[31], altered[31])

plt.imshow(dilated_gray ,cmap = 'gray')